__Tables (Skills)__  
  
0 - Overall  
1 - Attack  
2 - Defence  
3 - Strength  
4 - Hitpoints  
5 - Ranged  
6 - Prayer  
7 - Magic  
8 - Cooking  
9 - Woodcutting  
10 - Fletching  
11 - Fishing  
12 - Firemaking  
13 - Crafting  
14 - Smithing  
15 - Mining  
16 - Herblore  
17 - Agility  
18 - Thieving  
19 - Slayer  
20 - Farming  
21 - Runecraft  
22 - Hunter  
23 - Construction  

In [1]:
import requests
import pandas
import sqlite3 as sql
from bs4 import BeautifulSoup
import pyspark
from pyspark.sql.types import DateType
from datetime import datetime

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("osrs")\
    .getOrCreate()

#conf = pyspark.SparkConf()
#sc = pyspark.SparkContext(conf.setAppName("osrs"))

#sc = pyspark.SparkContext("osrs")
#sqlContext = pyspark.sql.SQLContext(spark)
#sc = pyspark.SparkContext
#sc.broadcast(url, 'https://secure.runescape.com/m=hiscore_oldschool/index_lite.ws?player=')

In [2]:
df = spark.read.load('data/player_names.parquet').repartition(8)
#rdd = df.rdd

In [24]:
rdd.getNumPartitions()

8

In [8]:
players = df.select('name').limit(5)
players.collect()

[Row(name='IDK Hunter'),
 Row(name='Evil rake'),
 Row(name='Hoibur'),
 Row(name='madnesh'),
 Row(name='AGM-86')]

In [8]:
def get_single_player(players):
    from datetime import datetime
    #print(player['name'])
    # base url for api request
    player_url_base = 'https://secure.runescape.com/m=hiscore_oldschool/index_lite.ws?player='
    
    for player in players:
        name = player['name']
        url = player_url_base + name
    
        now = datetime.utcnow()
        time = '{0}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}'.format(now.year, now.month, now.day, now.hour, now.minute, now.second)
        
        try:
            page = requests.get(url).text.replace(u'\n', u' ')
            skills = [i.split(',') for i in page.split()]
            xp = [i[2] for i in skills[0:24]]
            rank = [i[0] for i in skills[0:24]]
        except:
            print(name)
            continue
       
    return (name, time, xp, rank)

In [30]:
def get_single_player_part(players):
    from datetime import datetime
    #print(player['name'])
    # base url for api request
    player_url_base = 'https://secure.runescape.com/m=hiscore_oldschool/index_lite.ws?player='
    
    players_data = []
    
    for player in players:
        name = player['name']
        url = player_url_base + name
    
        now = datetime.utcnow()
        time = '{0}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}'.format(now.year, now.month, now.day, now.hour, now.minute, now.second)
        
        try:
            page = requests.get(url).text.replace(u'\n', u' ')
            skills = [i.split(',') for i in page.split()]
            xp = [i[2] for i in skills[0:24]]
            rank = [i[0] for i in skills[0:24]]
            players_data.append(tuple((name, time, xp, rank)))
        except:
            print(name)
            continue
            
    yield players_data

In [48]:
players = df.select('name').repartition(8)
players_hs = players.rdd.mapPartitions(get_single_player_part)
players_hs.is_checkpointed = True
print('Checkpoint enabled: ', players_hs.is_checkpointed,'\nNum partitions: ', players_hs.getNumPartitions())

Checkpoint enabled:  True 
Num partitions:  8


In [49]:
print(datetime.utcnow())
players_results = players_hs.collect()
print(datetime.utcnow())

2019-05-23 23:03:50.139876
2019-05-23 23:05:02.642486


In [35]:
len(players_results[0])

3

In [28]:
name = 'Tyler'
xp = 100
skill = 'att'
tuple((name, xp, skill))

('Tyler', 100, 'att')

In [42]:
total = 0
for i in range(len(players_results)):
    total += len(players_results[i])
total

20

In [6]:
rdd.__init__

NameError: name 'rdd' is not defined

In [47]:
rdd = players.rdd
rdd.getNumPartitions()

4

In [48]:
rdd2 = sc.parallelize(rdd, 10)

AttributeError: 'RDD' object has no attribute 'defaultParallelism'

In [49]:
rdd.checkpoint

<bound method RDD.checkpoint of MapPartitionsRDD[45] at javaToPython at NativeMethodAccessorImpl.java:0>

In [36]:
num = sc.accumulator(1)
def f(x):
    global num
    num+=x
rdd = sc.parallelize([20, 30, 40, 50])
rdd.foreach(f)
final = num.value
print('acc value is: {0}'.format(final))

TypeError: accumulator() missing 1 required positional argument: 'value'

In [83]:
a = [players.collect()][0]
a

[Row(name='IDK Hunter'),
 Row(name='Evil rake'),
 Row(name='Hoibur'),
 Row(name='madnesh'),
 Row(name='AGM-86'),
 Row(name='NoBotsPlease'),
 Row(name='Intu'),
 Row(name='Mr Carl'),
 Row(name='Dark Zatana'),
 Row(name='Jerry PvMs')]

In [ ]:
def get_player(conn, skill):
    # base url for api request
    player_url_base = 'https://secure.runescape.com/m=hiscore_oldschool/index_lite.ws?player='
    
    skills = ["Overall", "Attack", "Defence", "Strength", "Hitpoints", "Ranged", "Prayer", "Magic", "Cooking",
              "Woodcutting", "Fletching", "Fishing", "Firemaking", "Crafting", "Smithing", "Mining", "Herblore",
              "Agility", "Thieving", "Slayer", "Farming", "Runecraft", "Hunter", "Construction"]
    skill_name = skills[skill]
    print(skill_name)
    
    players = conn.execute('''SELECT name from players_name WHERE skill = (?)''', (skill_name,)).fetchall()
    num_players = len(players)
    
    now = datetime.utcnow()
    time = '{0}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}'.format(now.year, now.month, now.day, 
                                                                now.hour, now.minute, now.second)
    failed = []
    num = 0
    
    for player in players:
        # sql returns list of tuples, this isolates the name
        name = player[0]
        url = player_url_base + name
        
        num += 1
        if not num%20:
            print('Players scraped: {0}/{1}'.format(i, num_players), datetime.utcnow() - now)
        else:
            pass
        
        try:
            page = requests.get(url).text.replace(u'\n', u' ')
            skills = [i.split(',') for i in page.split()]
            xp = [i[2] for i in skills[0:24]]
            rank = [i[0] for i in skills[0:24]]
        except:
            print(name)
            failed.append(name)
            continue
            
        conn.execute('''INSERT INTO players_xp VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', [name, time] + xp)
        conn.execute('''INSERT INTO players_rank VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', [name, time] + rank)
            
    return failed

In [3]:
df.schema

StructType(List(StructField(name,StringType,true),StructField(date,DateType,true),StructField(skill,StringType,true),StructField(rank,LongType,true)))

In [35]:
from pyspark.sql.types import DateType
df = df.withColumn('date', df['date'].cast(DateType()))

In [42]:
df.write.mode('overwrite').parquet('data/player_names.parquet')

Py4JJavaError: An error occurred while calling o166.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:198)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:159)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope

In [20]:
spark.sql('use data/osrs.db')

ParseException: "\nmismatched input '/' expecting <EOF>(line 1, pos 8)\n\n== SQL ==\nuse data/osrs.db\n--------^^^\n"

In [21]:
import pyspark as spark
spark.sql("use data/osrs.db")

TypeError: 'module' object is not callable

In [3]:
#subtract 3 from line number to get table number, 0-23
#e.g., table 9 is woodcutting
skills = ["Overall",
          "Attack",
          "Defence",
          "Strength",
          "Hitpoints",
          "Ranged",
          "Prayer"
          "Magic",
          "Cooking",
          "Woodcutting",
          "Fletching",
          "Fishing",
          "Firemaking",
          "Crafting",
          "Smithing",
          "Mining",
          "Herblore",
          "Agility",
          "Thieving",
          "Slayer",
          "Farming",
          "Runecraft",
          "Hunter",
          "Construction"]
skills

['Overall',
 'Attack',
 'Defence',
 'Strength',
 'Hitpoints',
 'Ranged',
 'PrayerMagic',
 'Cooking',
 'Woodcutting',
 'Fletching',
 'Fishing',
 'Firemaking',
 'Crafting',
 'Smithing',
 'Mining',
 'Herblore',
 'Agility',
 'Thieving',
 'Slayer',
 'Farming',
 'Runecraft',
 'Hunter',
 'Construction']

Get player names and insert into osrs.db

In [122]:
def create_players_table(conn):
    
    conn.execute('''CREATE TABLE IF NOT EXISTS players(
                    "name" TEXT PRIMARY KEY,
                    "date" DATETIME NOT NULL,
                    "skill" TEXT,
                    "rank" INTEGER)''')
    conn.commit()

In [195]:
def scrape_names(min_rank, max_rank, skill):
    import requests
    import sqlite3 as sql
    from datetime import datetime
    from bs4 import BeautifulSoup
    import hs_tables
    
    conn = sql.connect('data/osrs.db')
    
    # create players table if it doesn't already exist
    # attributes: name, date, skill
    hs_tables.create_players_table(conn)
    
    # get skill name for inserting attribute in datebase table
    skills = ["Overall", "Attack", "Defence", "Strength", "Hitpoints", "Ranged", "Prayer", "Magic", "Cooking",
              "Woodcutting", "Fletching", "Fishing", "Firemaking", "Crafting", "Smithing", "Mining", "Herblore",
              "Agility", "Thieving", "Slayer", "Farming", "Runecraft", "Hunter", "Construction"]
    skill_name = skills[skill]
    
    # time in utc for later time series analysis
    now = datetime.utcnow()
    time = '{0}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}'.format(now.year, now.month, now.day, 
                                                                now.hour, now.minute, now.second)
    
    total_base = 'https://secure.runescape.com/m=hiscore_oldschool/overall.ws?table={0}&page='.format(skill)
    #player_url_base = 'https://secure.runescape.com/m=hiscore_oldschool/index_lite.ws?player='
    
    # range of pages to scrape results
    min_page = min_rank//25
    max_page = max_rank//25
    pages = max_page - min_page
    #url_no_page = ranking_url_base + 'table={22}&page='.format(table_num)
    
    for i in range(min_page, max_page):
        
        url = total_base + str(i)
        soup = BeautifulSoup(requests.get(url).text)
        try:
            table = soup.find("tbody")
        # create catch here
        except:
            return soup
        
        for row in table.findAll("tr"):
            player = row.findAll("td")
            # cols = [rank, name, level, xp]
            cols = [element.text.strip() for element in player]
            #name = cols[1].replace(u'\xa0', u' ')
            
            if cols[1]:
                name = cols[1].replace(u'\xa0', u' ')
                #print(name)
                rank = int(cols[0].replace(u',', u''))
                try:
                    conn.execute('''INSERT OR IGNORE INTO players(name, date, skill, rank) VALUES (?,?,?,?)''',(name, time, skill_name, int(rank)))
                    conn.commit()
                except:
                    print('Failed:', cols)
                    continue
            
            else:
                continue
            
        if not i%20:
            print('Pages scraped: {0}/{1}'.format(i-min_page, pages), datetime.utcnow() - now)
        else:
            continue
            
    return table

In [7]:
try:
    import importlib
    importlib.reload(scrape_hs)
    print('Reimported scrape_hs.py')
except:
    import scrape_hs

# 1400 end, 35000 done
min_rank = 485000 #440000 min originally
max_rank = 561000
skill = 23 # construction = 23

table = scrape_hs.scrape_names(min_rank, max_rank, skill)

Pages scraped: 0/3040 2019-05-23 00:42:32.248702
Pages scraped: 10/3040 2019-05-23 00:42:52.417858
Pages scraped: 90/3040 2019-05-23 00:47:09.248114
Pages scraped: 110/3040 2019-05-23 00:48:39.260316
Pages scraped: 130/3040 2019-05-23 00:50:06.607790
Pages scraped: 150/3040 2019-05-23 00:51:33.753334
Pages scraped: 160/3040 2019-05-23 00:52:19.528423
Pages scraped: 180/3040 2019-05-23 00:53:51.589447
Pages scraped: 250/3040 2019-05-23 00:58:54.393519
Pages scraped: 280/3040 2019-05-23 01:01:07.680285
Pages scraped: 300/3040 2019-05-23 01:02:33.434284
Pages scraped: 310/3040 2019-05-23 01:03:18.902874
Pages scraped: 430/3040 2019-05-23 01:12:08.331737
Pages scraped: 440/3040 2019-05-23 01:12:48.781933
Pages scraped: 450/3040 2019-05-23 01:13:35.068626
Pages scraped: 470/3040 2019-05-23 01:15:06.100727
Pages scraped: 480/3040 2019-05-23 01:15:51.974203
Pages scraped: 500/3040 2019-05-23 01:17:18.609096
Pages scraped: 600/3040 2019-05-23 01:24:39.643897
Pages scraped: 620/3040 2019-05-23 

In [8]:
conn.commit()
conn.close()

In [25]:
a = conn.execute('''SELECT name from players_name''').fetchall()
len(a)

47365

In [4]:
conn.execute('''CREATE TABLE IF NOT EXISTS players_name(
                    "name" TEXT, 
                    "date" DATETIME NOT NULL, 
                    "skill" TEXT, 
                    "rank" INTEGER)''')
print('1')


1


In [5]:
conn.execute('''PRAGMA table_info("players_name")''').fetchall()

[(0, 'name', 'TEXT', 0, None, 0),
 (1, 'date', 'DATETIME', 1, None, 0),
 (2, 'skill', 'TEXT', 0, None, 0),
 (3, 'rank', 'INTEGER', 0, None, 0)]

In [6]:
url = 'https://secure.runescape.com/m=hiscore_oldschool/overall.ws?table=23&page=2000'
requests.get(url).text[:100]

'\n<!doctype html>\n<!--[if lt IE 7]><html class="no-js lt-ie10 lt-ie9 lt-ie8 lt-ie7" lang="en"><![endi'

In [12]:
import pandas as pd
df = pd.DataFrame({'name': [],
             'date': [],
             'skill': [],
             'rank': []})
df

,name,date,skill,rank


In [17]:
df = df.append({'name': 'Tyler', 'date': 'now', 'skill': 'abc', 'rank': 100}, ignore_index=True)
df

,name,date,skill,rank
0,Tyler,now,abc,100.0
1,Tyler,now,abc,100.0


In [182]:
conn.execute('''SELECT * from players''').fetchall()

[('Hellz Zerker', '2019-05-22 19:34:35', 'Construction', 439976),
 ('Grant2k1', '2019-05-22 19:34:35', 'Construction', 439977),
 ('Kyle SR', '2019-05-22 19:34:35', 'Construction', 439978),
 ('Supreme_v1', '2019-05-22 19:34:35', 'Construction', 439979),
 ('M3EK MlLLY', '2019-05-22 19:34:35', 'Construction', 439980),
 ('Von Disney', '2019-05-22 19:34:35', 'Construction', 439981),
 ('ChrisW247', '2019-05-22 19:34:35', 'Construction', 439982),
 ('CleanedTotal', '2019-05-22 19:34:35', 'Construction', 439983),
 ('buga shuga', '2019-05-22 19:34:35', 'Construction', 439984),
 ('OBG Nicola', '2019-05-22 19:34:35', 'Construction', 439985),
 ('M d m a zing', '2019-05-22 19:34:35', 'Construction', 439986),
 ('MrTrunks', '2019-05-22 19:34:35', 'Construction', 439987),
 ('Logless', '2019-05-22 19:34:35', 'Construction', 439988),
 ('thafamilia3', '2019-05-22 19:34:35', 'Construction', 439989),
 ('SSGvegito', '2019-05-22 19:34:35', 'Construction', 439990),
 ('Space Mankey', '2019-05-22 19:34:35', 'Con

In [179]:
conn.close()
conn = sql.connect('data/osrs.db')

conn.execute('''DROP TABLE players''')

In [164]:
import time
time.time()

1558553079.964841

In [43]:
def create_hs_tables(conn):
    
    conn.execute('''CREATE TABLE IF NOT EXISTS player_xp (
                    name TEXT PRIMARY KEY,
                    date DATETIME NOT NULL,
                    overall INTEGER,
                    attack INTEGER,
                    defence INTEGER,
                    strength INTEGER,
                    hitpoints INTEGER,
                    ranged INTEGER,
                    prayer INTEGER,
                    magic INTEGER,
                    cooking INTEGER,
                    woodcutting INTEGER,
                    fletching INTEGER,
                    fishing INTEGER,
                    firemaking INTEGER,
                    crafting INTEGER,
                    smithing INTEGER,
                    mining INTEGER,
                    herblore INTEGER,
                    agility INTEGER,
                    thieving INTEGER,
                    slayer INTEGER,
                    farming INTEGER,
                    runecraft INTEGER,
                    hunter INTEGER,
                    construction INTEGER)''')

    conn.execute('''CREATE TABLE IF NOT EXISTS player_rank (
                    name TEXT PRIMARY KEY,
                    date DATETIME NOT NULL,
                    overall INTEGER,
                    attack INTEGER,
                    defence INTEGER,
                    strength INTEGER,
                    hitpoints INTEGER,
                    ranged INTEGER,
                    prayer INTEGER,
                    magic INTEGER,
                    cooking INTEGER,
                    woodcutting INTEGER,
                    fletching INTEGER,
                    fishing INTEGER,
                    firemaking INTEGER,
                    crafting INTEGER,
                    smithing INTEGER,
                    mining INTEGER,
                    herblore INTEGER,
                    agility INTEGER,
                    thieving INTEGER,
                    slayer INTEGER,
                    farming INTEGER,
                    runecraft INTEGER,
                    hunter INTEGER,
                    construction INTEGER)''')

(['MattsMauler',
  'Zen Santa',
  'Dr McGroober',
  'Bird Brained',
  'BoxOfCondums',
  'Brother Boof',
  'Typulsion',
  'Boyo Bob',
  'High Pro',
  'Emperator',
  'Mabaza',
  'Tokkurainen',
  'Akaaay',
  'leathalchick',
  'KinG CarrotI',
  'Truitedulac3',
  'Lightworks',
  'Lukeed113',
  '100 hp',
  'Sushi Phil',
  'HappinessLTD',
  'RevToo',
  'zdog520',
  'Embezzlez',
  'ICESKATE'],
 ['440,000', 'ICESKATE', '51', '112,073'])

In [69]:
import datetime
now = datetime.datetime.utcnow()
now

datetime.datetime(2019, 5, 22, 18, 5, 33, 113844)

In [72]:
now.second

33

In [75]:
time = '{0}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}'.format(now.year, now.month, now.day, now.hour, now.minute, now.second)
time

'2019-05-22 18:05:33'

In [64]:
from datetime import date
today = datetime.datetime.utcnow()
today

datetime.datetime(2019, 5, 22, 18, 3, 43, 340375)

In [64]:
# table_num from skills list above, e.g., table 9 = woodcutting
def scrape_rank(table_num, num_players):
    
    conn = sql.connect('osrs_hs.db')
    
    max_pages = num_players//25
    url_no_page = ranking_url_base + 'table={0}&page='.format(table_num)
    
    for i in range(max_pages):
        url = url_no_page + str(i)
        soup = BeautifulSoup(requests.get(url).text)
        try:
            table = soup.find("tbody")
        except:
            return players
        
        for row in table.findAll("tr"):
            player = row.findAll("td")
            cols = [element.text.strip() for element in player]
            players.append(cols[1])
            
    return players

In [4]:
# define base URLs for later queries
# ranking URL ends in: table={0}&page={1} where table number is in order of skills list
ranking_url_base = 'https://secure.runescape.com/m=hiscore_oldschool/overall.ws?' 
player_url_base = 'https://secure.runescape.com/m=hiscore_oldschool/index_lite.ws?player='



